In [ ]:
# TO DO: 
# - add CI as variable into bootstrap methods
# - normalize data and rerun
# - try on other data distribution

In [ ]:
import os
import datetime
import numpy as np
import math
import pandas as pd
from scipy import stats
from scipy.stats import ttest_ind, chisquare
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from math import cos, asin, sqrt
from time import time
sns.set(color_codes=True)
from datetime import datetime as dt

# pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 300)
# pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_seq_items', 999)
import plotly.graph_objects as go

In [ ]:
df_all = pd.read_csv('yellow_tripdata_2019-06.csv', usecols = ['tpep_pickup_datetime', 'total_amount'], parse_dates=['tpep_pickup_datetime'])
# df_all = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-06.csv')
# https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
# https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-06.csv
df_all


In [ ]:
df_all.describe()

In [ ]:
# there are some outliers, remove them. In addition, leave oonly positive amounts

In [ ]:
outliers_limit = df_all.total_amount.quantile(0.999)

In [ ]:
df_all = df_all[df_all.total_amount.between(0,outliers_limit, inclusive=False)]

In [ ]:
df_all.total_amount.plot.hist()

In [ ]:
df_all['date'] = df_all['tpep_pickup_datetime'].dt.normalize()

In [ ]:
# remove not june data
df_all = df_all[df_all.date.between('2019-06-01', '2019-06-30', inclusive=True)]

In [ ]:
df_all.shape

In [ ]:
# all bootstrapping will be done on subset of data

df = df_all.sample(frac=0.05).reset_index(drop=True).rename(columns={'total_amount':'metric'})
df.shape

In [ ]:
df.metric.plot.hist()

plt.title('Metric distribution')
plt.show()

In [ ]:
# full year data set
# df_all = pd.read_csv('2018_Yellow_Taxi_Trip_Data.csv')
# https://data.cityofnewyork.us/Transportation/2018-Yellow-Taxi-Trip-Data/t29m-gskq


In [ ]:
# groups = 3
# df['group'] = 'a'
# df.loc[round(df.shape[0])*0.2:, 'group'] = 'b'
# df.loc[round(df.shape[0])*0.6:, 'group'] = 'c'

In [ ]:
# groups = 2
# df['group'] = 'a'
# df.loc[round(df.shape[0])*0.5:, 'group'] = 'b'

In [ ]:
# df.groupby('group').metric.count()

In [ ]:
# implement https://ocw.mit.edu/courses/mathematics/18-05-introduction-to-probability-and-statistics-spring-2014/readings/MIT18_05S14_Reading24.pdf

In [ ]:
# averages
# df.groupby('group').metric.mean()

In [ ]:
df_all.total_amount.describe()

In [ ]:
df.metric.describe()

In [ ]:
ttest_ind(df_all.total_amount,df.metric)

In [ ]:
sns.distplot(df_all.total_amount)
sns.distplot(df.metric)
plt.title('Compare distributions')
plt.show()

### Estimate the mean μ of the underlying distribution and give an 95% bootstrap confidence interval.


In [ ]:
# df_a = df[df.group=='a']

obs_mean = df.metric.mean()
obs_mean

In [ ]:
# generate 100 bootstrap samples, each of size df, compute mean for each

n_bootstraps = 100

# empirical (or pivotal)

def empirical_bootstrap(series, obs_mean, return_bounds=True, n_bootstraps = 100, random=100, verbose=False):
    means = []
    for random in range(n_bootstraps):
        bootstrap = series.sample(n=series.shape[0], replace=True, random_state=random)
        bootstrap_mean = bootstrap.mean()
        means.append(bootstrap_mean)

    means = np.array(means)
    # compute δ∗ = x∗ − x for each bootstrap sample (i.e. each column) and sort them from smallest to biggest:
    diff = means - obs_mean
    diff.sort()
    if verbose:
        plt.hist(diff)
        plt.show()
        
    percentile_upper = np.percentile(diff, 2.5)
    percentile_lower = np.percentile(diff, 97.5)
#     print(percentile_upper, percentile_lower)
    empirical_lower = obs_mean - percentile_lower
    empirical_upper = obs_mean - percentile_upper
    if return_bounds:
        return(empirical_lower, empirical_upper)
    else:
        return(percentile_upper*(-1), percentile_lower)

In [ ]:
empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100)

In [ ]:
def percentile_bootstrap(series, obs_mean, return_bounds=True, n_bootstraps = 100, random=100):
    means = []
    for random in range(n_bootstraps):
        bootstrap = series.sample(n=series.shape[0], replace=True, random_state=random)
        bootstrap_mean = bootstrap.mean()
        means.append(bootstrap_mean)

    means = np.array(means)
    means.sort()
    percentile_lower = np.percentile(means, 2.5)
    percentile_upper = np.percentile(means, 97.5)
    if return_bounds:
        return(percentile_lower, percentile_upper)
    else:
        return(obs_mean-percentile_lower, percentile_upper-obs_mean)

In [ ]:
# compare with percentile bootstrap
percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100, random=100)

In [ ]:
def calculate_ci(sample, confidence = 0.95):
    n = sample.count()
    stdev = sample.std()
    test_stat = stats.norm.ppf((confidence + 1)/2)
    standard_error = test_stat * stdev / math.sqrt(n)
    return standard_error


In [ ]:
ci_lower = df.metric.mean() - calculate_ci(df.metric)
ci_upper = df.metric.mean() + calculate_ci(df.metric)
ci_lower, ci_upper

In [ ]:
# normal bootstrap (how we usually do. The only difference that I take mean as mean of bootstrap runs, not observational
def ci_bootstrap(series, n_bootstraps = 100, return_bounds = True, random=100):
    means = []
    for random in range(n_bootstraps):
        bootstrap = series.sample(n=series.shape[0], replace=True, random_state=random)
        bootstrap_mean = bootstrap.mean()
        means.append(bootstrap_mean)

    means = np.array(means)
    ci_lower = means.mean() - calculate_ci(pd.Series(means))
    ci_upper = means.mean() + calculate_ci(pd.Series(means))
    if return_bounds:
        return(means.mean(), ci_lower, ci_upper)
    else:
        return(means.mean(), calculate_ci(pd.Series(means)), calculate_ci(pd.Series(means)))

In [ ]:
ci_bootstrap(df.metric, return_bounds=False)

In [ ]:
# # plt.bar(x=1, height = obs_mean)
# plt.errorbar(x=1, y = obs_mean, yerr=[[lower], [-1*upper]], fmt='o', color='green')
# plt.errorbar(x=2, y = obs_mean, yerr=[[obs_mean-lower_p], [upper_p-obs_mean]], fmt='o', color='green')
# plt.errorbar(x=3, y = obs_mean, yerr=[[calculate_ci(pd.Series(means))], [calculate_ci(pd.Series(means))]], fmt='o', color='green')
# plt.show()

In [ ]:
population_mean = df_all.total_amount.mean()
methods = ['empirical', 'percentile', 'normal', 'ci', 'population_mean']
means = [obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean, population_mean]

lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100)[0],
        percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100, random=100)[0],
        ci_bootstrap(df.metric, return_bounds=False)[1],
        calculate_ci(df.metric),
        calculate_ci(df_all.total_amount)]

upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100)[1],
        percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = 100, random=100)[1],
        ci_bootstrap(df.metric, return_bounds=False)[2],
        calculate_ci(df.metric),
        calculate_ci(df_all.total_amount)]


In [ ]:
results = pd.DataFrame([methods, means, lower, upper]).transpose()
results.columns = ['method', 'mean', 'lower', 'upper']
results

In [ ]:
fig = go.Figure(data=go.Scatter( # Bar or Scatter
        x=methods,
        y=means,
        error_y=dict(
            type='data', 
            symmetric=False,
            array=upper,
            arrayminus=lower)
    )
                , layout ={ "title": f'Bootstrap Samples = {n_bootstraps}'
                           ,"width": 640
#                            , "yaxis": {"range": [19.5,20]}
                          }
                           )

fig.show()

In [ ]:
# TO DO
# compare how it decreases with incresing bootstrap numbers
# another approach to take random

### Experiment with bootsrtap sample numbers

In [ ]:
# try different bootsrtap sample numbers
for n_bootstraps in [50, 100, 200, 500, 1000]:

    means = [obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean]

    lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[0],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[0],
            ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
            calculate_ci(df.metric)]

    upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[1],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[1],
            ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
            calculate_ci(df.metric)]
    fig = go.Figure(data=go.Scatter( # Bar or Scatter
        y=methods,
        x=means,
            error_x=dict(
                type='data', 
                symmetric=False,
                array=upper,
                arrayminus=lower)
        )
                    , layout ={ "title": f'Bootstrap Samples = {n_bootstraps}'
                               ,"width": 640
    #                            , "yaxis": {"range": [19.5,20]}
                              }
                               )

    fig.show()
    results = pd.DataFrame([methods, means, lower, upper]).transpose()
    results.columns = ['method', 'mean', 'lower', 'upper']
    display(results)

### Experiment with different sample size

In [ ]:
# experiment with different sample size
for size in [50, 100, 500, 1000, 10000, 20000, 50000]:
    n_bootstraps = 500
    df = df_all.sample(n=size).reset_index(drop=True).rename(columns={'total_amount':'metric'})
    obs_mean = df.metric.mean()
    
    means = [obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean]

    lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[0],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[0],
            ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[1],
            calculate_ci(df.metric)]

    upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps)[1],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=100)[1],
            ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps)[2],
            calculate_ci(df.metric)]
    fig = go.Figure(data=go.Scatter( # Bar or Scatter
        y=methods,
        x=means,
            error_x=dict(
                type='data', 
                symmetric=False,
                array=upper,
                arrayminus=lower)
        )
                    , layout ={ "title": f'Samples in data = {size}'
                               ,"width": 640
                               , "xaxis": {"range": [13,24]}
                              }
                               )

    fig.show()
    results = pd.DataFrame([methods, means, lower, upper]).transpose()
    results.columns = ['method', 'mean', 'lower', 'upper']
    display(results)

### Experiment with random state

In [ ]:
# try different bootsrtap sample numbers
for random in [1,4,7,134, 7653, 2456]:
    n_bootstraps = 500
    df = df_all.sample(frac = 0.05).reset_index(drop=True).rename(columns={'total_amount':'metric'})
    obs_mean = df.metric.mean()
    means = [obs_mean, obs_mean, ci_bootstrap(df.metric)[0], obs_mean]

    lower = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=random)[0],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=random)[0],
            ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps, random=random)[1],
            calculate_ci(df.metric)]

    upper = [empirical_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=random)[1],
            percentile_bootstrap(df.metric, obs_mean, return_bounds=False, n_bootstraps = n_bootstraps, random=random)[1],
            ci_bootstrap(df.metric, return_bounds=False, n_bootstraps=n_bootstraps, random=random)[2],
            calculate_ci(df.metric)]
    fig = go.Figure(data=go.Scatter( # Bar or Scatter
        y=methods,
        x=means,
            error_x=dict(
                type='data', 
                symmetric=False,
                array=upper,
                arrayminus=lower)
        )
                    , layout ={ "title": f'Random State = {random}'
                               ,"width": 640
                               , "xaxis": {"range": [19.3,20]}
                              }
                               )

    fig.show()
    results = pd.DataFrame([methods, means, lower, upper]).transpose()
    results.columns = ['method', 'mean', 'lower', 'upper']
    display(results)

In [ ]:
df_all['total_amount'].mean() - calculate_ci(pd.Series(df_all['total_amount'])), df_all['total_amount'].mean() + calculate_ci(pd.Series(df_all['total_amount']))

In [ ]:
19.6053-0.0494632,19.6053+0.0482493

In [ ]:
19.6038-0.0021766,19.6038+0.0021766